In [ ]:
#Regression code
# Load data
final_data <- read.csv("data.csv")
dim_data <- read.csv("dim_data.csv")

# #################################################### Linear Regression Plot ####################################################
# Standardize features first
features <- c( "id", "BMI", "HR", "SBP", "DBP", "hba1c", "cr", "hdl", "ldl", "tg", "alt")
feature_table <- final_data[features]
hosp <- feature_table$hosp
id <- feature_table$id
feature_table <- feature_table[,-c(1)]  # Remove 'hosp' and 'id' columns for analysis

#rank_matrix <- apply(feature_table, 1, rank)  # Rank each feature row-wise
rank_matrix <- as.matrix(rank_matrix)

# Residualize data
for (i in 1:nrow(rank_matrix)){
  model <- lm(rank_matrix[i,] ~ final_data$GENDER + final_data$age)
  rank_matrix[i,] <- model$residuals  # Replace row with residuals
}

# Combine residualized data with IDs
processed_data <- cbind(hosp, id, t(rank_matrix))

# Merge residualized data with dimension coordinates
regression_data <- merge(processed_data, dim_data, by = c("id"))

######## Batch linear regression for multiple variables, no modifications needed
# Load necessary libraries
library(tidyverse)
library(rms)

# Define regression function
regression2 <- function(x) {
  # Fit linear model adjusting for hk_test_dim1 and hk_test_dim2
  model <- lm(x ~ hk_test_dim1 + hk_test_dim2, data = regression_data)
  
  # Extract model coefficients and confidence intervals
  coefficients <- coef(model)
  conf_intervals <- confint(model)
  
  # Convert to data frame
  coefficients_df <- as.data.frame(coefficients)
  coefficients_df$CI_low <- conf_intervals[, 1]
  coefficients_df$CI_high <- conf_intervals[, 2]
  
  return(coefficients_df)
}

# Apply regression to each variable and organize results
result <- map(regression_data[, c(3:12)], regression2) %>%
  do.call(rbind, .) %>%
  as.data.frame() %>%
  rownames_to_column("trait")

# Remove rows with Intercept term
result <- result %>% filter(!str_detect(trait, "Intercept"))

# Assign variable names and Dimension labels
variable_names <- c("BMI", "HR", "SBP", "DBP", "HbA1c", "CREAT", "HDL-C", "LDL-C", "TRIG", "ALT")
result <- result %>%
  mutate(
    name = rep(variable_names, each = 2),
    trait = rep(c("Dimension1", "Dimension2"), times = length(variable_names))
  ) %>%
  rename(beta = coefficients)

# Rearrange columns for final output table
final_table <- result %>% select(name, trait, beta, CI_low, CI_high)

# Display and save final results table
print(final_table)
write.csv(final_table, "linear regression.csv")

# ################# Plotting Linear Regression, no modifications needed #################
library(ggplot2)
final_table <- read.csv("linear regression.csv")

final_plot <- ggplot(final_table, aes(x = beta, y = reorder(name, beta), color = trait)) +
  geom_point(size = 3) +  # Plot regression coefficients
  geom_errorbarh(aes(xmin = CI_low, xmax = CI_high), height = 0.35) +  # Plot confidence intervals
  geom_vline(xintercept = 0, linetype = "dashed", color = "black") +  # Add zero line
  labs(title = "Regression Coefficients with 95% CI",
       x = "Regression Coefficient",
       y = "Model",
       color = "Dimension") +
  theme_minimal()

# Display plot
print(final_plot)

# Save plot
ggsave("figure_linear_regression.png", plot = final_plot, width = 8, height = 6, dpi = 300)

    term = rep(c("Dimension1", "Dimension2"), times = length(variable_names))
  )
write.csv(final_results, "cox regression.csv")


 

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns

def compareABunchOfDifferentModelsAccuracy(X_train, y_train):
    """
    Compare multiple classifiers using 10-fold cross-validation
    """
    print('\nCompare Multiple Classifiers: \n')
    print('K-Fold Cross-Validation Accuracy: \n')

    models = {
        'LR': LogisticRegression(),
        'RF': RandomForestClassifier(),
        'KNN': KNeighborsClassifier(),
        'SVM': SVC(),
        'LSVM': LinearSVC(),
        'GNB': GaussianNB(),
        'DTC': DecisionTreeClassifier(),
        'GBC': GradientBoostingClassifier(),
        'LASSO (L1)': LogisticRegressionCV(
            Cs=10, cv=10, penalty='l1', solver='saga', max_iter=10000, scoring='accuracy', refit=True
        ),
        'Elastic Net': LogisticRegressionCV(
            Cs=10, cv=10, penalty='elasticnet', solver='saga',
            l1_ratios=[.1, .5, .9], max_iter=10000,
            scoring='accuracy', refit=True
        )
    }

    names = []
    resultsAccuracy = []

    for name, model in models.items():
        kfold = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        resultsAccuracy.append(accuracy_results)
        names.append(name)
        print(f"{name}: {accuracy_results.mean():.4f} (+/- {accuracy_results.std():.4f})")

    # 可视化 boxplot
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=resultsAccuracy, palette='viridis')
    plt.xticks(ticks=range(len(names)), labels=names, rotation=45)
    plt.ylabel('Accuracy Score')
    plt.title('Comparison of Classifier Accuracy (10-fold CV)')
    plt.tight_layout()
    plt.show()

def defineModels():
    print('LR = LogisticRegression')
    print('RF = RandomForestClassifier')
    print('KNN = KNeighborsClassifier')
    print('SVM = Support Vector Machine SVC')
    print('LSVM = LinearSVC')
    print('GNB = GaussianNB')
    print('DTC = DecisionTreeClassifier')
    print('GBC = GradientBoostingClassifier')
    print('LASSO (L1) = LogisticRegressionCV with L1')
    print('Elastic Net = LogisticRegressionCV with ElasticNet\n\n')

data = read_csv('....csv')
X = data.iloc[:, 1:]  # except diabetes= X
y = data.iloc[:, 0]   # diabetes=y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train3 = pd.DataFrame(X_train)
print(y_train.dtype)  
print(y_train.unique())  

compareABunchOfDifferentModelsAccuracy(X_train, y_train)
defineModels()
import matplotlib.pyplot as plt

save_path = r".tiff"
plt.savefig(save_path, dpi=300, format='tiff', bbox_inches='tight')

#
plt.show()

print(f"figure saved: {save_path}")


In [ ]:
#SUPFig4
from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# 
models = {
    'LR': LogisticRegression(max_iter=1000),
    'RF': RandomForestClassifier(n_estimators=100),
    'SVM': SVC(kernel='rbf', probability=True),
    'LinearSVM': LinearSVC(max_iter=10000),
    'GBC': GradientBoostingClassifier(),
    'GNB': GaussianNB(),
    'DTC': DecisionTreeClassifier(),
    'LASSO': Lasso(alpha=0.01, max_iter=10000),
    'ElasticNet': ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=10000)
}

print("\nModel Comparison: ROC AUC and PR AUC\n")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)
    print(f"{name:20s} | ROC AUC: {roc_auc:.3f} | PR AUC: {pr_auc:.3f}")
    
plt.figure(figsize=(12, 5))
# ROC Curve
plt.subplot(1, 2, 1)
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_score = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc_score:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for All Models')
plt.legend()

# PR Curve
plt.subplot(1, 2, 2)
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)
    plt.plot(recall, precision, label=f"{name} (AUC={pr_auc:.2f})")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for All Models')
plt.legend()

plt.tight_layout()
plt.show()

plt.rcParams.update({'font.family': 'Arial', 'font.size': 12})
plt.savefig("model_comparison_auc.png", dpi=300, bbox_inches='tight')

In [ ]:
#Compare GBC and RF 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.metrics import accuracy_score

# Step 1: defination
feature_sets = {
    "Top 5": ["HDL-C", "TG", "SBP", "ALT", "HbA1c"],
    "Top 6": ["HDL-C", "TG", "SBP", "ALT", "HbA1c", "LDL-C"],
    "Top 7": ["HDL-C", "TG", "SBP", "ALT", "HbA1c", "LDL-C", "HR"],
    "Top 8": ["HDL-C", "TG", "SBP", "ALT", "HbA1c", "LDL-C", "HR", "BMI"],
    "Top 9": ["HDL-C", "TG", "SBP", "ALT", "HbA1c", "LDL-C", "HR", "BMI", "DBP"],
    "Top 10": ["HDL-C", "TG", "SBP", "ALT", "HbA1c", "LDL-C", "HR", "BMI", "DBP", "CREAT"]
}

# Step 2: read data
data = pd.read_csv('/ahsl.csv')

# Step 3: 
models = {
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42)
}

results = []

for model_name, model in models.items():
    for set_name, features in feature_sets.items():
        X = data[features]
        y = data['diabetes']
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=1, stratify=y)
        
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)

        roc = roc_auc_score(y_test, y_proba)
        pr = average_precision_score(y_test, y_proba)
        f1 = f1_score(y_test, y_pred)

        results.append({
            "Model": model_name,
            "Features": set_name,
            "ROC AUC": roc,
            "PR AUC": pr,
            "F1 Score": f1
        })

# Step 4: DataFrame
results_df = pd.DataFrame(results)
print(results_df)

#Visualize SUPfig4
import seaborn as sns

metrics = ["ROC AUC", "PR AUC", "F1 Score"]

for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=results_df, x="Features", y=metric, hue="Model", marker="o")
    plt.title(f"{metric} vs Number of Features")
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
#####Fig6F sankey
import pandas as pd

import plotly.graph_objects as go

# read data
data = pd.read_excel('....xlsx')

nodes_ordered = list(pd.unique(data['source'].tolist() + data['target'].tolist()))

node_indices = {node: i for i, node in enumerate(nodes_ordered)}

data['source'] = data['source'].map(node_indices)
data['target'] = data['target'].map(node_indices)

node_labels = nodes_ordered

sankey_fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=node_labels,
    ),
    link=dict(
        source=data['source'],  # 
        target=data['target'],  # 
        value=data['value'],    # 
    )
))

sankey_fig.update_layout(title_text="Energy", title_x=0.5)

sankey_fig.write_html("sankey_plot_0801001.html")

In [ ]:
#####SUPFig13 metabolic change
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_excel('...0922.xlsx')

# clean NaN
df_clean = df.dropna()

# delta 
delta_columns = ['delta_bmi', 'delta_sbp', 'delta_dbp', 'delta_hr', 'delta_hba1c',
                 'delta_cr', 'delta_tcho', 'delta_hdl', 'delta_ldl', 'delta_alt', 'delta_ast', 'delta_glu']

# set 3x4 
fig, axes = plt.subplots(3, 4, figsize=(20, 15))

fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, delta_var in enumerate(delta_columns):
    row = i // 4
    col = i % 4
    heatmap_data = df.pivot(index="Source", columns="Target", values=delta_var)
    sns.heatmap(heatmap_data, annot=True, cmap="coolwarm", linewidths=0.5, ax=axes[row, col])
    axes[row, col].set_title(f'{delta_var} change')

plt.tight_layout()
plt.show()

In [ ]:
##SUPFig14
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# dataframe
groups = ['Group1','Group2','Group3','Group4','Group5','Group6']

# complications
complications = ['MI', 'Stroke', 'I stroke', 'IH stroke', 'HF', 'DR', 'CKD', 'ESRD', 'PD', 'MASLD', 'Cirrhosis']

# contain Group1~6
data_raw = {
    ('Group1','Group1'): [6,27,24,1,13,16,17,5,5,53,6],
    ('Group1','Group2'): [1,4,4,1,4,3,0,0,0,10,1],
    ('Group1','Group3'): [1,0,0,0,1,2,0,0,1,2,0],
    ('Group1','Group4'): [0,0,0,0,0,0,0,0,0,0,0],
    ('Group1','Group5'): [0,1,1,0,1,0,0,0,0,0,0],
    ('Group1','Group6'): [0,5,5,1,2,7,3,1,1,16,2],
    
    ('Group2','Group1'): [1,4,4,1,2,5,0,0,5,12,1],
    ('Group2','Group2'): [3,7,4,1,8,6,4,5,3,20,1],
    ('Group2','Group3'): [0,0,0,0,1,0,0,0,0,1,0],
    ('Group2','Group4'): [0,0,0,0,0,0,0,0,0,1,1],
    ('Group2','Group5'): [0,0,0,0,0,0,1,1,0,0,0],
    ('Group2','Group6'): [0,4,4,0,1,3,2,0,1,0,2],
    
    ('Group3','Group1'): [0,3,2,0,1,1,0,0,0,1,0],
    ('Group3','Group2'): [0,0,0,0,1,0,0,0,0,1,0],
    ('Group3','Group3'): [1,3,2,1,2,2,4,1,2,8,1],
    ('Group3','Group4'): [0,0,0,2,0,0,1,0,0,0,0],
    ('Group3','Group5'): [0,0,0,0,0,0,0,0,0,0,0],
    ('Group3','Group6'): [0,0,0,0,0,0,0,0,0,0,0],
    
    ('Group4','Group1'): [0,0,0,0,0,0,0,0,0,0,0],
    ('Group4','Group2'): [0,0,0,0,0,0,0,0,0,0,0],
    ('Group4','Group3'): [0,1,1,0,0,0,0,0,0,0,0],
    ('Group4','Group4'): [0,0,0,0,1,1,1,0,1,2,1],
    ('Group4','Group5'): [1,2,2,0,2,1,1,0,0,1,1],
    ('Group4','Group6'): [0,0,0,0,0,0,0,0,0,0,0],
    
    ('Group5','Group1'): [0,1,0,0,1,2,1,0,0,2,0],
    ('Group5','Group2'): [0,1,1,0,0,1,0,0,1,1,0],
    ('Group5','Group3'): [0,0,0,0,0,0,0,0,0,0,0],
    ('Group5','Group4'): [1,1,1,0,1,1,1,0,1,4,3],
    ('Group5','Group5'): [0,14,10,1,7,10,7,4,0,53,1],
    ('Group5','Group6'): [2,10,9,0,4,7,4,2,3,24,2],
    
    ('Group6','Group1'): [0,8,8,0,6,5,7,3,0,16,2],
    ('Group6','Group2'): [0,2,2,0,1,1,1,1,1,1,0],
    ('Group6','Group3'): [0,0,0,0,0,1,0,0,0,1,1],
    ('Group6','Group4'): [0,0,0,0,0,1,0,0,0,1,1],
    ('Group6','Group5'): [0,6,6,1,1,4,1,1,1,12,0],
    ('Group6','Group6'): [4,29,27,4,14,25,12,3,4,76,4],
}

# DataFrame
index = pd.MultiIndex.from_tuples(data_raw.keys(), names=['From', 'To'])
df = pd.DataFrame(list(data_raw.values()), index=index, columns=complications)
df = df.loc[(slice('Group1','Group6'), slice('Group1','Group6')), :]

# heatmap
plt.figure(figsize=(18, 25))
for i, comp in enumerate(complications):
    plt.subplot(6, 2, i+1)
    pivot_table = df[comp].unstack(level='To')
    sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlGnBu', cbar_kws={'label': 'Events'})
    plt.title(f'Events of {comp} by Group Transition')
    plt.xlabel('To Group')
    plt.ylabel('From Group')

plt.tight_layout()
plt.show()

In [ ]:
#Chinese mapping function
#################################################################### First Part: Calculate Coordinates and Generate Feature Plots ##########################################################
setwd("china tree mapping function")

# Load necessary data
load("china_dim.RData")
load("Chinese_gam_x.rds")
load("Chinese_gam_y.rds")

##################################### The function definition below does not need modification ######################################################
ddrtree_map <- function(data) {
  library(mgcv)
  library(ggplot2)
  
  # Prepare the data
  names(data)[1] <- 'GENDER'
  names(data)[2] <- 'age'
  names(data)[3] <- 'hdl'
  names(data)[4] <- 'ldl'
  names(data)[5] <- 'tg'
  names(data)[6] <- 'hba1c'
  names(data)[7] <- 'BMI'
  names(data)[8] <- 'SBP'
  names(data)[9] <- 'DBP'
  names(data)[10] <- 'alt'
  names(data)[11] <- 'cr'
  names(data)[12] <- 'HR'
  
  print(names(data))
  
  # Data type conversion
  data[, c(2:12)] <- apply(data[, c(2:12)], 2, function(x) as.numeric(as.character(x)))
  data$GENDER <- factor(data$GENDER, levels = c('1', '2'))
  
  # Get the position prediction
  dim1 <- predict.gam(Chinese_gam_x, newdata = data)
  dim2 <- predict.gam(Chinese_gam_y, newdata = data)
  
  # Initialize new columns
  predict_data <- as.data.frame(cbind(dim1, dim2))
  predict_data$hk_test_dim1 <- NA
  predict_data$hk_test_dim2 <- NA
  
  # Reposition prediction points based on the Euclidean distance
  for (i in 1:nrow(predict_data)) {
    distance <- ((china_dim$data_dim_1 - predict_data$dim1[i])^2 + (china_dim$data_dim_2 - predict_data$dim2[i])^2)^0.5
    j <- which.min(distance)
    predict_data$hk_test_dim1[i] <- china_dim$data_dim_1[j]
    predict_data$hk_test_dim2[i] <- china_dim$data_dim_2[j]
  }
  
  # Check structure of predict_data
  print(head(predict_data))
  
  new_predict_data <- as.data.frame(cbind(predict_data, data))
  
  # Define indicators and color settings
  indicators <- c("SBP","DBP","hdl","ldl","BMI","tg","cr","alt","hba1c","HR")  # Indicator column names
  color_settings <- list(
    SBP = list(color_title = "SBP", title = "SBP (mm of Hg)", limits = c(80, 200), breaks = c(100, 120, 140, 160, 180), labels = c("100", "120", "140", "160", "180"), low = "#00FFFF", high = "red"),
    DBP = list(color_title = "DBP", title = "DBP (mm of Hg)", limits = c(40, 140), breaks = c(60, 80, 100, 120), labels = c("60", "80", "100", "120"), low = "#00FFFF", high = "red"),
    hdl = list(color_title = "HDL-C", title = "HDL-C (mmol/L)", limits = c(0, 2), breaks = c(0.5, 1, 1.5), labels = c("0.5", "1", "1.5"), low = "#00FFFF", high = "red"),
    ldl = list(color_title = "LDL-C", title = "LDL-C (mmol/L)", limits = c(0, 5), breaks = c(2, 4), labels = c("2", "4"), low = "#00FFFF", high = "red"),
    BMI = list(color_title = "BMI", title = "BMI (Kg/m2)", limits = c(20, 30), breaks = c(22, 24, 26, 28), labels = c("22", "24", "26", "28"), low = "#00FFFF", high = "red"),
    tg = list(color_title = "TRIG", title = "Triglycerides (mmol/L)", limits = c(0, 5), breaks = c(2, 4), labels = c("2", "4"), low = "#00FFFF", high = "red"),
    cr = list(color_title = "CREAT", title = "Creatinine (umol/L)", limits = c(50, 100), breaks = c(60, 70, 80, 90), labels = c("60", "70", "80", "90"), low = "#00FFFF", high = "red"),
    alt = list(color_title = "ALT", title = "ALT (IU/L)", limits = c(0, 75), breaks = c(15, 30, 45, 60), labels = c("15", "30", "45", "60"), low = "#00FFFF", high = "red"),
    hba1c = list(color_title = "HbA1c", title = "HbA1c (%)", limits = c(5, 12), breaks = c(6, 8, 10), labels = c("6", "8", "10"), low = "#00FFFF", high = "red"),
    HR = list(color_title = "HR", title = "HR (bpm)", limits = c(45, 135), breaks = c(60, 75, 90, 105, 120), labels = c("60", "75", "90", "105", "120"), low = "#00FFFF", high = "red")
  )
  
  create_plot <- function(indicator) {
    settings <- color_settings[[indicator]]
    
    # Base layer
    k <- ggplot(china_dim, aes(x = data_dim_1, y = data_dim_2)) +
      geom_point(color = 'grey') +  # Original data points in grey
      xlab('Dimension 1') +
      ylab('Dimension 2') +
      theme_minimal() +
      labs(title = settings$title) +
      theme(plot.title = element_text(size = 22, face = "bold"))
    
    # Add new layer on top of prediction points
    final_plot <- k +
      geom_point(data = new_predict_data, aes_string(x = 'hk_test_dim1', y = 'hk_test_dim2', color = indicator), size = 2) +
      scale_color_gradient(
        limits = settings$limits,
        breaks = settings$breaks,
        labels = settings$labels,
        low = settings$low,
        high = settings$high,
        oob = scales::squish
      ) +
      labs(color = settings$color_title) +
      theme(legend.text = element_text(size = 15),
            legend.title = element_text(size = 16))
    
    return(final_plot)
  }
  
  # Return the predicted data
  return(list(predict_data = predict_data, create_plot = create_plot, indicators = indicators))
}

###########################################################################
# Read data, modify the file path as necessary
data <- read.csv("sampledata.csv")

data2 <- data[, !names(data) %in% "id"] #remove col"id"

# Call ddrtree_map function to get predictions, no modifications needed
result <- ddrtree_map(data2)

# Get predicted data, no modifications needed
predicted_data <- result$predict_data
new_data <- cbind(data, predicted_data)

# Save the data, modify the file path as necessary
write.csv(new_data, "new_data.csv")

# Generate and save the plots for each indicator, modify the file path as necessary
for (indicator in result$indicators) {
  plot <- result$create_plot(indicator)
  ggsave(paste0("/ahsl/", indicator, "_plot.png"), plot, width = 8, height = 6)
}

#################################################################### Second Part: Competing Risk Model to Calculate Complication Risk Probability ########################
outcome_data <- read.csv("sampleoutcome.csv")  # Load the dataset containing outcome data

final_data <- merge(outcome_data, new_data, by = "id")  # Merge with the dataset containing the calculated coordinates, ensuring accurate matching

# Load necessary packages
library(cmprsk)
library(rms)

################################ General Function: Calculate survival time and cumulative incidence probability, no need to modify #####################
calculate_survival_and_prediction <- function(final_data, outcome_name, outcome_date_col, risk_col, crr_model_covariates) {
  # Calculate survival time
  survival_time_col <- paste0("survival_time_", outcome_name)
  final_data[[survival_time_col]] <- as.numeric((as.Date(final_data[[outcome_date_col]]) - as.Date(final_data$visit_start_date)) / 365)
  
  # Calculate risk variable
  risk_col_name <- paste0("risk_", outcome_name)
  final_data[[risk_col_name]] <- ifelse(final_data[[risk_col]] == 0 & final_data$dead == 0, 0, 
                                        ifelse(final_data[[risk_col]] == 1 & final_data$dead == 0, 1, 2))
  
  # Check for missing values in survival time and status, and filter data
  valid_data <- final_data[!is.na(final_data[[survival_time_col]]) & !is.na(final_data[[risk_col_name]]), ]
  
  # Create Cox regression model (crr)
  crr_model <- crr(ftime = valid_data[[survival_time_col]],
                   fstatus = valid_data[[risk_col_name]],
                   cov1 = as.matrix(valid_data[, crr_model_covariates]),
                   cencode = 0,
                   failcode = 1)
  
  # Output model summary
  print(summary(crr_model))
  
  # Get cumulative incidence probabilities from the fitted model
  predictions <- predict(crr_model, cov1 = as.matrix(valid_data[, crr_model_covariates]))
  
  # Get cumulative incidence probability at the last time point
  last_time_point_predictions <- predictions[, ncol(predictions)]
  
  # Format prediction results
  predictions_formatted <- format(last_time_point_predictions, digits = 4)
  
  # Create result column and add prediction results to the full dataset
  probability_col_name <- paste0(outcome_name, "_probability")
  final_data[[probability_col_name]] <- NA  # Initialize the result column
  final_data[[probability_col_name]][!is.na(final_data[[survival_time_col]]) & !is.na(final_data[[risk_col_name]])] <- predictions_formatted
  
  return(final_data)
}

################## Define outcome names, corresponding date columns, and risk columns (modify based on actual data) ################
outcome_list <- list(
  list("name" = "MAFLD", "date_col" = "MAFLD_date", "risk_col" = "MAFLD"),
  list("name" = "Cirrhosis", "date_col" = "Cirrhosis_date", "risk_col" = "Cirrhosis"),
  list("name" = "MI", "date_col" = "MI_date", "risk_col" = "MI"),
  list("name" = "Stroke", "date_col" = "Stroke_date", "risk_col" = "Stroke"),
  list("name" = "Istroke", "date_col" = "Istroke_date", "risk_col" = "Istroke"),
  list("name" = "IHstroke", "date_col" = "IHstroke_date", "risk_col" = "IHstroke"),
  list("name" = "HF", "date_col" = "HF_date", "risk_col" = "HF"),
  list("name" = "HFrEF", "date_col" = "HFrEF_date", "risk_col" = "HFrEF"),
  list("name" = "HFpEF", "date_col" = "HFpEF_date", "risk_col" = "HFpEF"),
  list("name" = "CKD", "date_col" = "CKD_date", "risk_col" = "CKD"),
  list("name" = "ESRD", "date_col" = "ESRD_date", "risk_col" = "ESRD"),
  list("name" = "DR", "date_col" = "DR_date", "risk_col" = "DR")
)

# No modification needed
crr_model_covariates <- c("hk_test_dim1", "hk_test_dim2")

# Loop through each outcome to calculate survival time and cumulative incidence probability. No modification needed. #########
for (outcome in outcome_list) {
  final_data <- calculate_survival_and_prediction(
    final_data,
    outcome_name = outcome$name,
    outcome_date_col = outcome$date_col,
    risk_col = outcome$risk_col,
    crr_model_covariates = crr_model_covariates
  )
}

# View the final merged dataset
colnames(final_data)
head(final_data)

write.csv(final_data, "final_data.csv")

####################################################### Third Part: Visualization ###################################################
final_data <- read.csv("final_data.csv")  # Load the saved final_data.csv

colnames(final_data)
library(ggplot2)
library(scales)
library(purrr)

### Modify based on actual data
indicators <- c("MAFLD_probability", "Cirrhosis_probability", "MI_probability", "Stroke_probability", "Istroke_probability", "IHstroke_probability", "HF_probability", "HFrEF_probability", "HFpEF_probability", "CKD_probability", "ESRD_probability", "DR_probability")  # Get indicator column names
color_settings <- list(
  MAFLD_probability = list(color_title = "Probability of MAFLD", title = "Probability of MAFLD",  low = "#00FFFF", high = "red"),
  Cirrhosis_probability = list(color_title = "Probability of Cirrhosis", title = "Probability of Cirrhosis", low = "#00FFFF", high = "red"),
  MI_probability = list(color_title = "Probability of MI", title = "Probability of MI",  low = "#00FFFF", high = "red"),
  Stroke_probability = list(color_title = "Probability of Stroke", title = "Probability of Stroke",  low = "#00FFFF", high = "red"),
  Istroke_probability = list(color_title = "Probability of Istroke", title = "Probability of Istroke", low = "#00FFFF", high = "red"),
  IHstroke_probability = list(color_title = "Probability of IHstroke", title = "Probability of IHstroke",  low = "#00FFFF", high = "red"),
  HF_probability = list(color_title = "Probability of HF", title = "Probability of HF",  low = "#00FFFF", high = "red"),
  HFrEF_probability = list(color_title = "Probability of HFrEF", title = "Probability of HFrEF", low = "#00FFFF", high = "red"),
  HFpEF_probability = list(color_title = "Probability of HFpEF", title = "Probability of HFpEF", low = "#00FFFF", high = "red"),
  CKD_probability = list(color_title = "Probability of CKD", title = "Probability of CKD", low = "#00FFFF", high = "red"),
  ESRD_probability = list(color_title = "Probability of ESRD", title = "Probability of ESRD", low = "#00FFFF", high = "red"),
  DR_probability = list(color_title = "Probability of DR", title = "Probability of DR",  low = "#00FFFF", high = "red")
)

############################### Define function, no need to modify #####################################
create_plot <- function(indicator) {
  settings <- color_settings[[indicator]]
  
  # Ensure the indicator column is numeric
  final_data[[indicator]] <- as.numeric(final_data[[indicator]])
  
  # Base plot
  k <- ggplot(china_dim, aes(x = data_dim_1, y = data_dim_2)) +
    geom_point(color = 'grey') +  # Original data points in grey
    xlab('Dimension 1') +
    ylab('Dimension 2') +
    theme_minimal() +
    labs(title = settings$title) +
    theme(plot.title = element_text(size = 22, face = "bold"))
  
  # Overlay new layer with prediction points
  final_plot <- k +
    geom_point(data = final_data, aes(x = hk_test_dim1, y = hk_test_dim2, color = !!sym(indicator)), size = 2) + 
    scale_color_gradient(
      low = settings$low,
      high = settings$high
    ) +
    labs(color = settings$color_title) +
    theme(legend.text = element_text(size = 15),
          legend.title = element_text(size = 16))
  
  return(final_plot)
}

#################################################################### Plotting #########
plots <- map(indicators, create_plot)
folder_path <- "/ashl/"  # Set the path to save the images

#########no need to modify #####################################
save_plot <- function(plot, indicator) {
  file.name <- paste0("plot_outcome_", indicator, ".png")
  file_path <- file.path(folder_path, file.name)
  ggsave(filename = file_path, plot = plot, width = 8, height = 6)
  cat("Plot saved as:", file_path, "\n")
}

walk2(plots, indicators, save_plot)
